In [1]:
import research_kit_bottom_trend_follow as rk
rk.pd.set_option("display.max_columns", 250)
rk.pd.set_option("display.max_rows", 250)
import warnings
warnings.filterwarnings('ignore')

downloading BTC/USD data on bitmex
downloading 2020-01-12 21:23:07.580862 ~ 2020-01-13 05:43:07.580862 data
4.8% completed
downloading 2020-01-13 05:43:07.580862 ~ 2020-01-13 14:03:07.580862 data
9.5% completed
downloading 2020-01-13 14:03:07.580862 ~ 2020-01-13 22:23:07.580862 data
14.3% completed
downloading 2020-01-13 22:23:07.580862 ~ 2020-01-14 06:43:07.580862 data
19.0% completed
downloading 2020-01-14 06:43:07.580862 ~ 2020-01-14 15:03:07.580862 data
23.8% completed
downloading 2020-01-14 15:03:07.580862 ~ 2020-01-14 23:23:07.580862 data
28.6% completed
downloading 2020-01-14 23:23:07.580862 ~ 2020-01-15 07:43:07.580862 data
33.3% completed
downloading 2020-01-15 07:43:07.580862 ~ 2020-01-15 16:03:07.580862 data
38.1% completed
downloading 2020-01-15 16:03:07.580862 ~ 2020-01-16 00:23:07.580862 data
42.9% completed
downloading 2020-01-16 00:23:07.580862 ~ 2020-01-16 08:43:07.580862 data
47.6% completed
downloading 2020-01-16 08:43:07.580862 ~ 2020-01-16 17:03:07.580862 data
52.4

downloading BTC/USD data on bitmex
downloading 2020-01-12 21:23:07.580862 ~ 2020-01-13 05:43:07.580862 data
4.8% completed
downloading 2020-01-13 05:43:07.580862 ~ 2020-01-13 14:03:07.580862 data
9.5% completed
downloading 2020-01-13 14:03:07.580862 ~ 2020-01-13 22:23:07.580862 data
14.3% completed
downloading 2020-01-13 22:23:07.580862 ~ 2020-01-14 06:43:07.580862 data
19.0% completed
downloading 2020-01-14 06:43:07.580862 ~ 2020-01-14 15:03:07.580862 data
23.8% completed
downloading 2020-01-14 15:03:07.580862 ~ 2020-01-14 23:23:07.580862 data
28.6% completed


KeyboardInterrupt: 

In [2]:
def show_bottom_trend_follow_params(bot):
    rk.pprint(bot.combined_params)

In [3]:
bot = rk.BottomTrendFollow(db_client=rk.mysql_client, exchange_client=rk.bitmex_exchange_client, is_backtest=True)
show_bottom_trend_follow_params(bot)

{'bot_name': 'bottom_trend_follow',
 'bottom_trend_tick': 12,
 'close_position_on_do_nothing': True,
 'inverse_trading': False,
 'middle_trend_tick': 6,
 'timeframe': 60,
 'top_trend_tick': 3}


In [4]:
def generate_transaction_log(bot, close_position_on_do_nothing=True, inverse_trading=False):
    backtest_start_time = rk.datetime(2019,9,30,11,49)
    backtest_end_time = rk.datetime(2019,12,29,11,49)
    
    ohlcv_df_1min = rk.dataset_manager.get_ohlcv(start_time=backtest_start_time, end_time=backtest_end_time)
    print("dataset size:" + str(len(ohlcv_df_1min)))
    
    timeframe_params = [60,30,10,1]
    bottom_trend_tick_params = [5,10,15,20,25,30,35,40,45,50]
    middle_trend_tick_params = [3,5,7,9,12,15,18,21,24,27,30]
    top_trend_tick_params = [1,2,3,4,5,6,7,8]
    
    calc_start_time = rk.datetime.now()
    
    for timeframe in timeframe_params:
        print("timeframe=>" + str(timeframe))
        for bottom_trend_tick in bottom_trend_tick_params:
            for middle_trend_tick in middle_trend_tick_params:           
                for top_trend_tick in top_trend_tick_params:
                    if bottom_trend_tick <= middle_trend_tick or middle_trend_tick <= top_trend_tick:
                        continue
                        
                    default_params = {
                        "bot_name": bot.bot_name,
                        "close_position_on_do_nothing": close_position_on_do_nothing,
                        "inverse_trading": inverse_trading,
                        "timeframe": int(timeframe)
                    }
                    
                    specific_params = {
                        "bottom_trend_tick": int(bottom_trend_tick),
                        "middle_trend_tick": int(middle_trend_tick),
                        "top_trend_tick": int(top_trend_tick)
                    }
                    
                    bot.reset_backtest_result_with_params(default_params, specific_params)
                    before_run = rk.datetime.now()
                    bot.run(ohlcv_df=ohlcv_df_1min[::timeframe], backtest_start_time=backtest_start_time,
                           backtest_end_time=backtest_end_time)
                    print("bottom_trend_tick=>" + str(bottom_trend_tick) +\
                          " midle_trend_tick=>" + str(middle_trend_tick) +\
                          " top_trend_tick=>" + str(top_trend_tick) + " time:" + str(rk.datetime.now() - before_run))
        bot.bulk_insert()
    print("total processing time:" + str(rk.datetime.now() - calc_start_time))

#generate_transaction_log(bot)
# it takes more than 4hours to calculate all wuth insert after getting transaction log before commit "optimize metrics calc"

#generate_transaction_log(bot, inverse_trading=False)
#generate_transaction_log(bot, inverse_trading=True)
# it takes 1hour + 10 min to caluculate the first one after update.
# it takes 1hour + 40 min to caluculate the second one after update